Table 3.2.1: Interventions received by young people in treatment in 2016-17 (pre November 2013 dataset change interventions)

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/statistics/substance-misuse-treatment-for-young-people-statistics-2016-to-2017')
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda t: t.startswith('Data tables')).as_databaker()}

In [2]:
tab = tabs['3.2.1 Interventions']

In [3]:
observations = tab.excel_ref('B4').expand(DOWN).expand(RIGHT).is_not_blank()
observations

{<B5 45.0>, <B4 '*'>}

In [4]:
intervention = tab.excel_ref('A4').expand(DOWN).is_not_blank() 
intervention

{<A4 'Inpatient detoxification'>, <A5 'Other YP intervention'>, <A7 '* All numbers under 5 have been suppressed. Where totals could be derived, figures have been rounded to the nearest 5 and marked with an asterisk. '>}

In [5]:
measuretype = tab.excel_ref('B3').expand(RIGHT).is_not_blank() 
measuretype

{<B3 'n'>}

In [6]:
Dimensions = [
            HDim(intervention,'Clients in treatment',DIRECTLY,LEFT),
            HDimConst('Basis of treatment','Change intervention'),
            HDim(measuretype,'Measure Type',DIRECTLY,ABOVE),
            HDimConst('Unit','People')            
            ]

In [7]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# if is_interactive():
#     savepreviewhtml(c1)

In [8]:
new_table = c1.topandas()
new_table

,OBS,DATAMARKER,Clients in treatment,Basis of treatment,Measure Type,Unit
0,,*,Inpatient detoxification,Change intervention,n,People
1,45,NaN,Other YP intervention,Change intervention,n,People


In [9]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [10]:
new_table.head()

,Value,DATAMARKER,Clients in treatment,Basis of treatment,Measure Type,Unit
0,,*,Inpatient detoxification,Change intervention,n,People
1,45,NaN,Other YP intervention,Change intervention,n,People


In [11]:
new_table['Measure Type'] = new_table['Measure Type'].map(
    lambda x: {
        'n' : 'Count', 
        '%' : 'Percentage',
        }.get(x, x))

In [12]:
new_table.tail()

,Value,DATAMARKER,Clients in treatment,Basis of treatment,Measure Type,Unit
0,,*,Inpatient detoxification,Change intervention,Count,People
1,45,NaN,Other YP intervention,Change intervention,Count,People


In [13]:
new_table.dtypes

Value                   object
DATAMARKER              object
Clients in treatment    object
Basis of treatment      object
Measure Type            object
Unit                    object
dtype: object

In [14]:
new_table['Value'] = new_table['Value'].astype(str)

In [15]:
new_table.head(3)

,Value,DATAMARKER,Clients in treatment,Basis of treatment,Measure Type,Unit
0,,*,Inpatient detoxification,Change intervention,Count,People
1,45.0,NaN,Other YP intervention,Change intervention,Count,People


In [16]:
new_table['Value'] = new_table['Value'].map(
    lambda x: {
        '' : '>5'
        }.get(x, x))

In [17]:
new_table['Period'] = '2016-17'
new_table['Substance'] = 'All'
new_table = new_table[['Period','Basis of treatment','Substance','Clients in treatment','Measure Type','Value','Unit']]

In [18]:
new_table.tail()

,Period,Basis of treatment,Substance,Clients in treatment,Measure Type,Value,Unit
0,2016-17,Change intervention,All,Inpatient detoxification,Count,>5,People
1,2016-17,Change intervention,All,Other YP intervention,Count,45.0,People
